# Natasha

Natasha solves basic NLP tasks for Russian language: tokenization, sentence segmentatoin, word embedding, morphology tagging, lemmatization, phrase normalization, syntax parsing, NER tagging, fact extraction.

Library is just a wrapper for lower level tools from <a href="https://github.com/natasha">Natasha project</a>:

* <a href="https://github.com/natasha/razdel">Razdel</a> — token, sentence segmentation for Russian
* <a href="https://github.com/natasha/navec">Navec</a> — compact Russian embeddings
* <a href="https://github.com/natasha/slovnet">Slovnet</a> — modern deep-learning techniques for Russian NLP, compact models for Russian morphology, syntax, NER.
* <a href="https://github.com/natasha/yargy">Yargy</a> — rule-based fact extraction similar to Tomita parser.
* <a href="https://github.com/natasha/ipymarkup">Ipymarkup</a> — NLP visualizations for NER and syntax markups.

Consider using these lower level tools for realword tasks. Natasha models are optimized for news articles, on other domains quality may be worse.

In [1]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

# Getting started

## Doc

`Doc` aggregates annotators, initially it has just `text` field defined:

In [2]:
text = 'Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры. Свое заявление он разместил в Twitter. «Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией. Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат. 11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года). В июле аналогичное решение принял Житомирский областной совет. В начале месяца с предложением к президенту страны Петру Порошенко вернуть Бандере звание Героя Украины обратились депутаты Верховной Рады. Парламентарии уверены, что признание Бандеры национальным героем поможет в борьбе с подрывной деятельностью против Украины в информационном поле, а также остановит «распространение мифов, созданных российской пропагандой». Степан Бандера (1909-1959) был одним из лидеров Организации украинских националистов, выступающей за создание независимого государства на территориях с украиноязычным населением. В 2010 году в период президентства Виктора Ющенко Бандера был посмертно признан Героем Украины, однако впоследствии это решение было отменено судом. '

doc = Doc(text)
doc

Doc(text='Посол Израиля на Украине Йоэль Лион признался, чт...)

After applying `segmenter` two new fields appear `sents` and `tokens`:

In [3]:
doc.segment(segmenter)
display(doc)
display(doc.sents[:2])
display(doc.tokens[:5])

Doc(text='Посол Израиля на Украине Йоэль Лион признался, чт..., tokens=[...], sents=[...])

[DocSent(stop=218, text='Посол Израиля на Украине Йоэль Лион признался, чт..., tokens=[...]),
 DocSent(start=219, stop=257, text='Свое заявление он разместил в Twitter.', tokens=[...])]

[DocToken(stop=5, text='Посол'),
 DocToken(start=6, stop=13, text='Израиля'),
 DocToken(start=14, stop=16, text='на'),
 DocToken(start=17, stop=24, text='Украине'),
 DocToken(start=25, stop=30, text='Йоэль')]

After applying `morph_tagger` and `syntax_parser`, tokens get 5 new fields `id`, `pos`, `feats`, `head_id`, `rel` — annotation in <a href="https://universaldependencies.org/">Universal Dependencies format</a>:

In [4]:
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
display(doc.tokens[:5])

[DocToken(stop=5, text='Посол', id='1_1', head_id='1_7', rel='nsubj', pos='NOUN', feats=<Anim,Nom,Masc,Sing>),
 DocToken(start=6, stop=13, text='Израиля', id='1_2', head_id='1_1', rel='nmod', pos='PROPN', feats=<Inan,Gen,Masc,Sing>),
 DocToken(start=14, stop=16, text='на', id='1_3', head_id='1_4', rel='case', pos='ADP'),
 DocToken(start=17, stop=24, text='Украине', id='1_4', head_id='1_1', rel='nmod', pos='PROPN', feats=<Inan,Loc,Fem,Sing>),
 DocToken(start=25, stop=30, text='Йоэль', id='1_5', head_id='1_1', rel='appos', pos='PROPN', feats=<Anim,Nom,Masc,Sing>)]

After applying `ner_tagger` doc gets `spans` field with PER, LOC, ORG annotation:

In [5]:
doc.tag_ner(ner_tagger)
display(doc.spans[:5])

[DocSpan(start=6, stop=13, type='LOC', text='Израиля', tokens=[...]),
 DocSpan(start=17, stop=24, type='LOC', text='Украине', tokens=[...]),
 DocSpan(start=25, stop=35, type='PER', text='Йоэль Лион', tokens=[...]),
 DocSpan(start=89, stop=106, type='LOC', text='Львовской области', tokens=[...]),
 DocSpan(start=152, stop=158, type='LOC', text='России', tokens=[...])]

## Visualizations

Natasha wraps <a href="https://github.com/natasha/ipymarkup">Ipymarkup</a> to provide ASCII visualizations for morphology, syntax and NER annotations. `doc` and `sents` have 3 methods: `morph.print()`, `syntax.print()` and `ner.print()`:

In [6]:
doc.ner.print()

Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав
      LOC────    LOC──── PER───────                                   
 о решении властей Львовской области объявить 2019 год годом лидера 
                   LOC──────────────                                
запрещенной в России Организации украинских националистов (ОУН) 
              LOC─── ORG─────────────────────────────────────── 
Степана Бандеры. Свое заявление он разместил в Twitter. «Я не могу 
PER────────────                                ORG────             
понять, как прославление тех, кто непосредственно принимал участие в 
ужасных антисемитских преступлениях, помогает бороться с 
антисемитизмом и ксенофобией. Украина не должна забывать о 
                              LOC────                      
преступлениях, совершенных против украинских евреев, и никоим образом 
не отмечать их через почитание их исполнителей», — написал дипломат. 
11 декабря Львовский областной совет принял решение провозгласить

In [8]:
sent = doc.sents[0]
sent.morph.print()

               Посол NOUN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
             Израиля PROPN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
                  на ADP
             Украине PROPN|Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing
               Йоэль PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
                Лион PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
           признался VERB|Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Mid
                   , PUNCT
                 что SCONJ
              пришел VERB|Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
                   в ADP
                 шок NOUN|Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
                   , PUNCT
               узнав VERB|Aspect=Perf|Tense=Past|VerbForm=Conv|Voice=Act
                   о ADP
             решении NOUN|Animacy=Inan|Case=Loc|Gender=Neut|Number=Sing
             властей NOUN|Animacy=Inan|Case=Gen|Gender

In [9]:
sent.syntax.print()

        ┌──► Посол         nsubj
        │    Израиля       
        │ ┌► на            case
        │ └─ Украине       
        │ ┌─ Йоэль         
        │ └► Лион          flat:name
┌─────┌─└─── признался     
│     │ ┌──► ,             punct
│     │ │ ┌► что           mark
│     └►└─└─ пришел        ccomp
│     │   ┌► в             case
│     └──►└─ шок           obl
│         ┌► ,             punct
│ ┌────►┌─└─ узнав         advcl
│ │     │ ┌► о             case
│ │ ┌───└►└─ решении       obl
│ │ │ ┌─└──► властей       nmod
│ │ │ │   ┌► Львовской     amod
│ │ │ └──►└─ области       nmod
│ └─└►┌─┌─── объявить      nmod
│     │ │ ┌► 2019          amod
│     │ └►└─ год           obj
│     └──►┌─ годом         obl
│   ┌─────└► лидера        nmod
│   │ ┌►┌─── запрещенной   acl
│   │ │ │ ┌► в             case
│   │ │ └►└─ России        obl
│ ┌─└►└─┌─── Организации   nmod
│ │     │ ┌► украинских    amod
│ │   ┌─└►└─ националистов nmod
│ │   │   ┌► (             punct
│ │   └►┌─└─ ОУН   

## Lemmatization

Tokens have `lemmatize` method, it uses `pos` and `feats` assigned by `morph_tagger` to get word normal form. `morph_vocab` is just a wrapper for <a href="https://pymorphy2.readthedocs.io/en/latest/">Pymorphy2</a>:

In [10]:
for token in doc.tokens:
    token.lemmatize(morph_vocab)
    
{_.text: _.lemma for _ in doc.tokens[:30]}

{'Посол': 'посол',
 'Израиля': 'израиль',
 'на': 'на',
 'Украине': 'украина',
 'Йоэль': 'йоэль',
 'Лион': 'лион',
 'признался': 'признаться',
 ',': ',',
 'что': 'что',
 'пришел': 'прийти',
 'в': 'в',
 'шок': 'шок',
 'узнав': 'узнать',
 'о': 'о',
 'решении': 'решение',
 'властей': 'власть',
 'Львовской': 'львовский',
 'области': 'область',
 'объявить': 'объявить',
 '2019': '2019',
 'год': 'год',
 'годом': 'год',
 'лидера': 'лидер',
 'запрещенной': 'запретить',
 'России': 'россия',
 'Организации': 'организация',
 'украинских': 'украинский',
 'националистов': 'националист'}

## Phrase normalization

Consider phrase "Организации украинских националистов", one can not just inflect every word independently to get normal form: "Организация украинский националист". Spans have method `normalize` that uses syntax annotation by `syntax_parser` to inflect phrases:

In [12]:
for span in doc.spans:
    span.normalize(morph_vocab)
    
{_.text: _.normal for _ in doc.spans}

{'Израиля': 'Израиль',
 'Украине': 'Украина',
 'Йоэль Лион': 'Йоэль Лион',
 'Львовской области': 'Львовская область',
 'России': 'Россия',
 'Организации украинских националистов (ОУН)': 'Организация украинских националистов (ОУН)',
 'Степана Бандеры': 'Степан Бандера',
 'Twitter': 'Twitter',
 'Украина': 'Украина',
 'Львовский областной совет': 'Львовский областной совет',
 'ОУН': 'ОУН',
 'Житомирский областной совет': 'Житомирский областной совет',
 'Петру Порошенко': 'Петр Порошенко',
 'Бандере': 'Бандера',
 'Украины': 'Украина',
 'Верховной Рады': 'Верховная Рада',
 'Бандеры': 'Бандера',
 'Степан Бандера': 'Степан Бандера',
 'Организации украинских националистов': 'Организация украинских националистов',
 'Виктора Ющенко': 'Виктор Ющенко',
 'Бандера': 'Бандера'}

## Fact extraction

To split names like "Виктор Ющенко", "Бандера" and "Йоэль Лион" into parts use `names_extractor` and spans method `extract_fact`:

In [13]:
for span in doc.spans:
    if span.type == PER:
        span.extract_fact(names_extractor)
    
{_.normal: _.fact.as_dict for _ in doc.spans if _.fact}

{'Йоэль Лион': {'first': 'Йоэль', 'last': 'Лион'},
 'Степан Бандера': {'first': 'Степан', 'last': 'Бандера'},
 'Петр Порошенко': {'first': 'Петр', 'last': 'Порошенко'},
 'Бандера': {'last': 'Бандера'},
 'Виктор Ющенко': {'first': 'Виктор', 'last': 'Ющенко'}}

# Reference

One may use Natasha components independently. It is not mandatory to construct `Doc` object.

## `Segmenter`

`Segmenter` is just a wrapper for <a href="https://github.com/natasha/razdel">Razdel</a>, it has two methods `tokenize` and `sentenize`:

In [14]:
tokens = list(segmenter.tokenize('Кружка-термос на 0.5л (50/64 см³, 516;...)'))
for token in tokens[:5]:
    print(token)

Token(start=0, stop=13, text='Кружка-термос')
Token(start=14, stop=16, text='на')
Token(start=17, stop=20, text='0.5')
Token(start=20, stop=21, text='л')
Token(start=22, stop=23, text='(')


In [15]:
# text = '''
# - "Так в чем же дело?" - "Не ра-ду-ют".
#  И т. д. и т. п. В общем, вся газета
# '''
sents = list(segmenter.sentenize(text))
for sent in sents:
    print(sent)

Sent(start=0, stop=218, text='Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры.')
Sent(start=219, stop=257, text='Свое заявление он разместил в Twitter.')
Sent(start=258, stop=424, text='«Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией.')
Sent(start=425, stop=592, text='Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат.')
Sent(start=593, stop=798, text='11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года).')
Sent(start=799, stop=861, text='В

## `MorphVocab`

`MorphVocab` is a wrapper for <a href="pymorphy2.readthedocs.io/en/latest/">Pymorphy2</a>. `MorphVocab` adds cache and adapts grammems to Universal Dependencies format:

In [16]:
forms = morph_vocab('стали')
forms

[MorphForm(normal='стать', pos='VERB', feats={'VerbForm': 'Fin', 'Aspect': 'Perf', 'Number': 'Plur', 'Tense': 'Past', 'Mood': 'Ind'}),
 MorphForm(normal='сталь', pos='NOUN', feats={'Animacy': 'Inan', 'Gender': 'Fem', 'Number': 'Sing', 'Case': 'Gen'}),
 MorphForm(normal='сталь', pos='NOUN', feats={'Animacy': 'Inan', 'Gender': 'Fem', 'Number': 'Sing', 'Case': 'Dat'}),
 MorphForm(normal='сталь', pos='NOUN', feats={'Animacy': 'Inan', 'Gender': 'Fem', 'Number': 'Sing', 'Case': 'Loc'}),
 MorphForm(normal='сталь', pos='NOUN', feats={'Animacy': 'Inan', 'Gender': 'Fem', 'Number': 'Plur', 'Case': 'Nom'}),
 MorphForm(normal='сталь', pos='NOUN', feats={'Animacy': 'Inan', 'Gender': 'Fem', 'Number': 'Plur', 'Case': 'Acc'})]

In [17]:
morph_vocab.__call__.cache_info()

CacheInfo(hits=239, misses=634, maxsize=10000, currsize=634)

Also `MorphVocab` adds method `lemmatize`. Given `pos` and `feats` it selects the most suitable morph form and returns its `normal` field:

In [18]:
morph_vocab.lemmatize('стали', 'VERB', {})

'стать'

In [19]:
morph_vocab.lemmatize('стали', 'X', {'Case': 'Gen'})

'сталь'

## `Embedding`

`Embedding` is a wrapper for <a href="https://github.com/natasha/navec/">Navec</a> — compact pretrained word embeddings for Russian language:

In [20]:
print('Words in vocab + 2 for pad and unk: %d' % len(emb.vocab.words) )

Words in vocab + 2 for pad and unk: 250002


In [27]:
emb['путин']

array([-4.27272737e-01,  6.87549114e-01,  4.40512747e-01,  2.17762515e-01,
       -1.95004210e-01,  1.73079148e-01,  9.91745573e-03,  4.18423414e-02,
       -6.39119446e-02, -5.02835929e-01,  7.31137246e-02, -4.38279688e-01,
       -2.33864665e-01,  3.85931171e-02,  6.10735156e-02,  8.62136409e-02,
        8.52477431e-01,  2.84970384e-02, -2.25771606e-01,  9.02989879e-02,
       -6.50790811e-01, -2.00142935e-02, -2.36029446e-01,  5.99771831e-03,
       -6.89393699e-01,  1.17030121e-01, -3.51317316e-01,  2.68402278e-01,
       -2.50863969e-01,  4.98451233e-01,  1.31768718e-01, -6.11520827e-01,
        3.98125112e-01, -2.25559071e-01,  7.71434456e-02, -8.18602592e-02,
       -8.47006738e-01, -4.94974077e-01,  3.14322025e-01, -4.87938374e-02,
        4.21800822e-01,  4.79611635e-01, -9.15634990e-01, -1.68828622e-01,
        4.04541194e-02,  5.06230481e-02, -4.72999290e-02, -9.65936407e-02,
       -6.27369881e-01, -2.34400466e-01,  2.58884966e-01, -3.20548624e-01,
       -5.63622117e-01, -

In [23]:
len(emb['путин'])

300

## `MorphTagger`

`MorphTagger` wraps <a href="https://github.com/natasha/slovnet">Slovnet morphology tagger</a>. Tagger has list of words as input and returns markup object. Markup has `print` method that outputs morph tags ASCII visualization:

In [22]:
words = ['Европейский', 'союз', 'добавил', 'в', 'санкционный', 'список', 'девять', 'политических', 'деятелей']
markup = morph_tagger(words)
markup.print()

         Европейский ADJ|Case=Nom|Degree=Pos|Gender=Masc|Number=Sing
                союз NOUN|Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
             добавил VERB|Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
                   в ADP
         санкционный ADJ|Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing
              список NOUN|Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
              девять NUM|Case=Nom
        политических ADJ|Case=Gen|Degree=Pos|Number=Plur
            деятелей NOUN|Animacy=Anim|Case=Gen|Gender=Masc|Number=Plur


## `SyntaxParser`

`SyntaxParser` wraps <a href="https://github.com/natasha/slovnet">Slovnet syntax parser</a>. Interface is similar to `MorphTagger`:

In [28]:
words = ['Европейский', 'союз', 'добавил', 'в', 'санкционный', 'список', 'девять', 'политических', 'деятелей']
markup = syntax_parser(words)
markup.print()

      ┌► Европейский  amod
    ┌►└─ союз         nsubj
  ┌─└─── добавил      
  │ ┌──► в            case
  │ │ ┌► санкционный  amod
┌─└►└─└─ список       obl
│   ┌──► девять       nummod:gov
│   │ ┌► политических amod
└──►└─└─ деятелей     nmod


## `NERTagger`

`NERTagger` wraps <a href="https://github.com/natasha/slovnet">Slovnet NER tagger</a>. Interface is similar to `MorphTagger` but has untokenized text as input:

In [29]:
text = 'Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры. Свое заявление он разместил в Twitter. 11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года).'
markup = ner_tagger(text)
markup.print()
print(markup.spans[2])
text[25:35]
print(markup.spans[1])
print(markup.spans[1].type, text[markup.spans[1].start:markup.spans[1].stop])

Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав
      LOC────    LOC──── PER───────                                   
 о решении властей Львовской области объявить 2019 год годом лидера 
                   LOC──────────────                                
запрещенной в России Организации украинских националистов (ОУН) 
              LOC─── ORG─────────────────────────────────────── 
Степана Бандеры. Свое заявление он разместил в Twitter. 11 декабря 
PER────────────                                ORG────             
Львовский областной совет принял решение провозгласить 2019 год в 
ORG──────────────────────                                         
регионе годом Степана Бандеры в связи с празднованием 110-летия со дня
              PER────────────                                         
 рождения лидера ОУН (Бандера родился 1 января 1909 года).
                 ORG                                      
Span(start=25, stop=35, type='PER')
Span(start=17, stop=24, 

## `Extractor`

In addition to `names_extractor` Natasha bundles several other extractors: `dates_extractor`, `money_extractor` and `addr_extractor`. All extractors are based on <a href="https://github.com/natasha/yargy">Yargy-parser</a>, meaning that they work correctly only on small predefined set of texts. For realword tasks consider writing your own grammar, see <a href="https://github.com/natasha/yargy#documentation">Yargy docs</a> for more.

### `DatesExtractor`

In [30]:
text = '24.01.2017, 2015 год, 2014 г, 1 апреля, май 2017 г., 9 мая 2017 года'
list(dates_extractor(text))

[Match(start=0, stop=10, fact=Date(year=2017, month=1, day=24)), Match(start=12, stop=20, fact=Date(year=2015, month=None, day=None)), Match(start=22, stop=28, fact=Date(year=2014, month=None, day=None)), Match(start=30, stop=38, fact=Date(year=None, month=4, day=1)), Match(start=40, stop=51, fact=Date(year=2017, month=5, day=None)), Match(start=53, stop=68, fact=Date(year=2017, month=5, day=9))]

### `MoneyExtractor`

In [31]:
text = '1 599 059, 38 Евро, 420 долларов, 20 млн руб, 20 т. р., 881 913 (Восемьсот восемьдесят одна тысяча девятьсот тринадцать) руб. 98 коп.'
list(money_extractor(text))

[Match(start=0, stop=18, fact=Money(amount=1599059.38, currency='EUR')), Match(start=20, stop=32, fact=Money(amount=420, currency='USD')), Match(start=34, stop=44, fact=Money(amount=20000000, currency='RUB')), Match(start=46, stop=54, fact=Money(amount=20000, currency='RUB')), Match(start=56, stop=133, fact=Money(amount=881913.98, currency='RUB'))]

### `NamesExtractor`

`names_extractor` should be applied only to spans of text. To extract single fact use method `find`:

In [32]:
lines = [
    'Отсутствующий Мустафа Джемилев увидел свет в конце',
    'О. Дерипаска',
    'Фёдор Иванович Шаляпин',
    'Янукович'
]
for line in lines:
    display(names_extractor.find(line))

Match(start=0, stop=21, fact=Name(first='Мустафа', last='Отсутствующий', middle=None))

Match(start=0, stop=12, fact=Name(first='О', last='Дерипаска', middle=None))

Match(start=0, stop=22, fact=Name(first='Фёдор', last='Шаляпин', middle='Иванович'))

Match(start=0, stop=8, fact=Name(first=None, last='Янукович', middle=None))

### `AddrExtractor`

In [35]:
lines = [
    'Россия, Вологодская обл. г. Череповец, пр.Победы 93 б, 692909, увидел с вышки РФ, Приморский край, г. Находка, ул. Добролюбова, 18',
#     '692909, РФ, Приморский край, г. Находка, ул. Добролюбова, 18',
    'ул. Народного Ополчения д. 9к.3'
]
for line in lines:
    display(addr_extractor.find(line))

Match(start=0, stop=126, fact=Addr(parts=[AddrPart(value='Россия', type='страна'), AddrPart(value='Вологодская', type='область'), AddrPart(value='Череповец', type='город'), AddrPart(value='Победы', type='проспект'), AddrPart(value='692909', type='индекс'), AddrPart(value='РФ', type='страна'), AddrPart(value='Приморский', type='край'), AddrPart(value='Находка', type='город'), AddrPart(value='Добролюбова', type='улица')]))

Match(start=0, stop=29, fact=Addr(parts=[AddrPart(value='Народного Ополчения', type='улица'), AddrPart(value='9к', type='дом')]))